# Image Preprocessing

OpenVINO provide [Optimize Preprocessing](https://docs.openvino.ai/2023.2/openvino_docs_OV_UG_Preprocessing_Overview.html) APIs.

#### Define benchmark function

In [ ]:
from time import time

import cv2
import numpy
import numpy as np
from ipywidgets import interact, widgets
from openvino.preprocess import ColorFormat, PrePostProcessor, ResizeAlgorithm
from openvino.runtime import Core, Type, Layout


def benchmark_model(infer_frames_func, batch_size):
    sec = 10
    count = 0
    start = time()
    inputs = np.random.randint(0, 256, size=(batch_size, 1080, 1920, 3), dtype=np.uint8)
    while time() - start < sec:
        infer_frames_func(inputs)
        count += batch_size

    print(f"FPS={count / sec:.2f}")


MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

#### Choose Model

In [ ]:
model_name = "resnet50"
model_type = "int8"

models = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152"]


@interact(option=widgets.Dropdown(options=models, value=model_name, description="model name"))
def read_dropdown(option):
    global model_name
    model_name = option


model_types = ["fp32", "fp16", "int8"]


@interact(option=widgets.Dropdown(options=model_types, value=model_type, description="model type"))
def read_dropdown(option):
    global model_type
    model_type = option


model_path = f"models/{model_name}/{model_type}/model.xml"

#### OpenCV Preprocess

In [ ]:
def opencv_preprocess():
    model = Core().compile_model(model_path, "CPU")
    req = model.create_infer_request()

    def infer_frames(input_frames: np.ndarray):
        mean = 255 * np.array(MEAN)
        std = 255 * np.array(STD)

        preprocessed_frames = []
        for frame in input_frames:
            frame = cv2.resize(frame, (224, 224))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame.transpose(2, 0, 1)  # HWC to CHW
            frame = (frame - mean[:, None, None]) / std[:, None, None]
            preprocessed_frames.append(frame)

        preprocessed_frames = np.stack(preprocessed_frames)
        req.infer(preprocessed_frames)

    return infer_frames

#### OpenCV Resize + OpenVINO Preprocess

In [ ]:
def opencv_resize_openvino_preprocess():
    model = Core().read_model(model_path)

    ppp = PrePostProcessor(model)
    ppp.input().tensor() \
        .set_element_type(Type.u8) \
        .set_layout(Layout('NHWC')) \
        .set_color_format(ColorFormat.BGR)

    ppp.input().model().set_layout(Layout('NCHW'))

    mean = 255 * numpy.array(MEAN)
    scale = 255 * numpy.array(STD)
    ppp.input().preprocess() \
        .convert_element_type(Type.f32) \
        .convert_color(ColorFormat.RGB) \
        .mean(mean) \
        .scale(scale)

    model = ppp.build()
    model = Core().compile_model(model, "CPU")
    req = model.create_infer_request()

    def infer_frames(input_frames: np.ndarray):
        preprocessed_frames = []
        for frame in input_frames:
            frame = cv2.resize(frame, (224, 224))
            preprocessed_frames.append(frame)

        preprocessed_frames = np.stack(preprocessed_frames)
        req.infer(preprocessed_frames)

    return infer_frames

#### OpenVINO Preprocess

In [ ]:
def openvino_preprocess():
    model = Core().read_model(model_path)

    ppp = PrePostProcessor(model)
    ppp.input().tensor() \
        .set_element_type(Type.u8) \
        .set_shape([-1, 1080, 1920, 3]) \
        .set_layout(Layout('NHWC')) \
        .set_color_format(ColorFormat.BGR)

    ppp.input().model().set_layout(Layout('NCHW'))

    mean = 255 * numpy.array(MEAN)
    scale = 255 * numpy.array(STD)
    ppp.input().preprocess() \
        .convert_element_type(Type.f32) \
        .convert_color(ColorFormat.RGB) \
        .resize(ResizeAlgorithm.RESIZE_LINEAR) \
        .mean(mean) \
        .scale(scale)

    model = ppp.build()
    model = Core().compile_model(model, "CPU")
    req = model.create_infer_request()

    def infer_frames(input_frames: np.ndarray):
        req.infer(input_frames)

    return infer_frames

#### Benchmark Result

Choose batch size

In [ ]:
bs = 8


@interact(batch_size="8")
def read_input(batch_size):
    global bs
    bs = int(batch_size)

1. OpenCV Preprocess + OpenVINO Inference

In [ ]:
benchmark_model(opencv_preprocess(), bs)

2. OpenCV Resize + OpenVINO Preprocess

In [ ]:
benchmark_model(opencv_resize_openvino_preprocess(), bs)

3. OpenVINO Preprocess

In [ ]:
benchmark_model(openvino_preprocess(), bs)